In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Prose/Mathieu/Benter-Project

/home/mathieu/Prose/Mathieu/Benter-Project


In [2]:
%matplotlib inline

import datetime as dt
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
import re
from itertools import combinations
import tensorflow as tf
import functools
from scipy.stats import rankdata
import scipy
import json

from utils import import_data
from winning_validation import errors
from winning_validation import r_squared
from winning_horse_models import sklearn
from winning_horse_models.dl_shared_layers import LogisticRegressionModel, DLSharedLayersModel
from winning_horse_models.xgboost import XGBoostWinningModel
from winning_horse_models.lgbm import LGBMWinningModel
from training_procedures import sequential_training, flattened_training
from constants import Sources
from utils import preprocess

from database.setup import create_sqlalchemy_session
from models.race import Race
from models.runner import Runner
tqdm.pandas()

/home/mathieu/.pyenv/versions/3.7.9/envs/benter-project_venv/lib/python3.7/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
SOURCE = Sources.UNIBET
N_FEATURES = preprocess.get_n_preprocessed_feature_columns(source=SOURCE)

In [4]:
# XGBoost

In [5]:
%%time
%%capture --no-stdout

#xgboost_winning_model, training_history = flattened_training.train_per_n_horses_races(source=SOURCE, winning_model=XGBoostWinningModel(source=SOURCE), verbose=True)
#xgboost_winning_model.save_model(prefix="48_col_")

CPU times: user 618 µs, sys: 194 µs, total: 812 µs
Wall time: 818 µs


In [6]:
xgboost_winning_model=XGBoostWinningModel.load_model(prefix="48_col_", source=SOURCE, n_features = N_FEATURES)

In [11]:
%%capture --no-stdout
%%time
res=r_squared.compute_mcfadden_r_squared_on_n_horses(source=SOURCE,winning_model=xgboost_winning_model, n_horses=10,verbose=False)

CPU times: user 8.75 s, sys: 106 ms, total: 8.86 s
Wall time: 8.24 s


In [13]:
res['model_r_squared']

-0.06956215250069908

In [ ]:
# Overfitting!

In [4]:
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [5]:
hyperparameters = dict(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)

In [11]:
# https://medium.com/analytics-vidhya/hyperparameter-tuning-hyperopt-bayesian-optimization-for-xgboost-and-neural-network-8aedf278a1c9
space={'max_depth': hp.quniform("max_depth", 2, 10, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.2,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.quniform('n_estimators', 20,180,1)
    }

In [12]:
   
# Classifier:
def hyperparameter_tuning(space):
    for param in ('max_depth', 'n_estimators', 'reg_alpha', 'min_child_weight'):
        if param in space:
            space[param] = int(space[param])
            
    for param in ('gamma', 'reg_lambda', 'colsample_bytree'):
        if param in space:
            space[param] = float(space[param])  
    model =XGBoostWinningModel(source=SOURCE, n_features = N_FEATURES, hyperparameters=space)
    
    model, _ = flattened_training.train_on_n_horses_races(source=SOURCE, winning_model=model, n_horses=10, verbose=True)

    res=r_squared.compute_mcfadden_r_squared_on_n_horses(source=SOURCE,winning_model=model, n_horses=10, verbose=True)
    r_squared__score= res['model_r_squared']
    print (f"R²: {r_squared__score:.2}, {space}")
    #change the metric if you like
    return {'loss': -r_squared__score, 'status': STATUS_OK, 'model': model}



In [ ]:
%%time
%%capture --no-stdout
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials,
           verbose=True)

print (best)

[07:51:08] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training for 10 horses (13625 races): loss per horse: 0.218, val loss per horse: 0.220 Train Accuracy: 23.4%, Val Accuracy: 20.9%

Comparing on same races w/ 10 horses with odds 2196 races (3182 races in total)
Mean Predicted probas of actual race result: 11.471% (Random: 9.999%, Odds: 20.823%)
On 2196 races with 10 horses,R² of winning model: 0.04, R² of odds: 0.19, [R² of random model: -0.13 (should be closed to 0)]
R²: 0.044, {'colsample_bytree': 0.867336526984638, 'gamma': 7.370804695882834, 'max_depth': 9, 'min_child_weight': 6, 'n_estimators': 33, 'reg_alpha': 48, 'reg_lambda': 0.6757482952936152}
[07:53:19] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:soft